---
title: Configure error mitigation
description: How to configure error mitigation with Qiskit Runtime.
---


# Configure error mitigation

<Admonition type="note" title="New execution model, now in beta release">
The beta release of a new execution model is now available. The directed execution model provides more flexibility when customizing your error mitigation workflow. See the [Directed execution model](/docs/guides/directed-execution-model) guide for more information.
</Admonition>

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.43.1
```
</details>

Error mitigation techniques allow users to mitigate circuit errors by
modeling the device noise at the time of execution. This typically
results in quantum pre-processing overhead related to model training and
classical post-processing overhead to mitigate errors in the raw results
by using the generated model.

The Estimator primitive supports several error mitigation techniques, including [TREX](/docs/api/qiskit-ibm-runtime/options-resilience-options-v2#measure_mitigation), [ZNE](/docs/api/qiskit-ibm-runtime/options-resilience-options-v2#zne), [PEC](/docs/api/qiskit-ibm-runtime/options-resilience-options-v2#pec), and [PEA](/docs/api/qiskit-ibm-runtime/options-zne-options). See [Error mitigation and suppression techniques](error-mitigation-and-suppression-techniques) for an explanation of each. When using primitives, you can turn on or off individual methods. See the [Custom error settings](#advanced-error) section for details.

<Admonition type="note">

Sampler does not support error mitigation, but you can use the [mthree](https://qiskit.github.io/qiskit-addon-mthree/) (matrix-free measurement mitigation) package to perform error mitigation locally.

</Admonition>

Estimator also supports `resilience_level`. The resilience level specifies how much resilience to build against
errors. Higher levels generate more accurate results, at the expense of
longer processing times. Resilience levels can be used to configure the
cost/accuracy trade-off when applying error mitigation to your primitive
query. Error mitigation reduces errors (bias) in results by processing
the outputs from a collection, or ensemble, of related circuits. The
degree of error reduction depends on the method applied. The resilience
level abstracts the detailed choice of error mitigation method to allow
users to reason about the cost/accuracy trade that is appropriate to
their application.

Given this, each level corresponds to a method or methods with
increasing level of quantum sampling overhead to enable you experiment
with different time-accuracy tradeoffs. The following table shows you
which levels and corresponding methods are available for each of the
primitives.

<Admonition type="info" title="Attention">
Error mitigation is task-specific, so the techniques you can
apply vary based whether you are sampling a distribution or generating
expectation values.
</Admonition>

<span id="resilience-table"></span>

Estimator supports the following resilience levels.  Sampler does not support resilience levels.

| Resilience Level | Definition                                                                                            | Technique                                                                 |
|------------------|-------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------|
| 0                | No mitigation                                                                                         | None                                                                      |
| 1 [Default]      | Minimal mitigation costs: Mitigate error associated with readout errors                               | Twirled Readout Error eXtinction  (TREX) measurement twirling             |
| 2                | Medium mitigation costs. Typically reduces bias in estimators, but is not guaranteed to be zero-bias. | Level 1 + Zero Noise Extrapolation (ZNE) and gate twirling


<Admonition type="info" title="Attention">

Resilience levels are currently in beta so sampling overhead and
solution quality will vary from circuit to circuit. New features,
advanced options, and management tools will be released on a rolling
basis. Specific error mitigation methods are not guaranteed to be
applied at each resilience level.
</Admonition>

## Configure Estimator with resilience levels

You can use resilience levels to specify error mitigation techniques, or you can set custom techniques individually as described in [Custom error settings.](#advanced-error)

<details>
<summary>Resilience Level 0</summary>

No error mitigation is applied to the user program.


</details>


<details>
<summary>Resilience Level 1</summary>

Level 1 applies **readout error mitigation** and **measurement twirling** by applying a model-free technique known
as Twirled Readout Error eXtinction (TREX). It reduces measurement error
by diagonalizing the noise channel associated with measurement by
randomly flipping qubits through X gates immediately before measurement. A
rescaling term from the diagonal noise channel is learned by
benchmarking random circuits initialized in the zero state. This allows
the service to remove bias from expectation values that result from
readout noise. This approach is described further in [Model-free
readout-error mitigation for quantum expectation
values](https://arxiv.org/abs/2012.09738).

</details>


<details>
<summary>Resilience Level 2</summary>

Level 2 applies the **error mitigation techniques included in level 1** and also applies **gate twirling** and uses the **Zero Noise Extrapolation method (ZNE)**.  ZNE computes an
expectation value of the observable for different noise factors
(amplification stage) and then uses the measured expectation values to
infer the ideal expectation value at the zero-noise limit (extrapolation
stage). This approach tends to reduce errors in expectation values, but
is not guaranteed to produce an unbiased result.


![This image shows a graph.  The x-axis is labeled Noise amplification factor.  The y-axis is labeled Expectation value.  An upward sloping line is labeled Mitigated value.  Points near the line are noise-amplified values.  There is a horizontal line just above the X-axis labeled Exact value.](/docs/images/guides/configure-error-mitigation/resilience-2.svg "Illustration of the ZNE method")

The overhead of this method scales with the number of noise factors. The
default settings sample the expectation value at three noise factors,
leading to a roughly 3x overhead when employing this resilience level.

In Level 2, the TREX method randomly flips qubits through X gates immediately before measurement,
and flips the corresponding measured bit if an X gate was applied. This approach is described further in [Model-free
readout-error mitigation for quantum expectation
values](https://arxiv.org/abs/2012.09738).

</details>

### Example

The `EstimatorV2` interface lets users seamlessly work with the variety of
error mitigation methods to reduce error in expectation values of
observables. The following code uses Zero Noise Extrapolation and readout error mitigation by simply
setting `resilience_level 2`.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Setting options during primitive initialization
estimator = Estimator(backend, options={"resilience_level": 2})

<span id="advanced-error"></span>
## Custom error settings

You can turn on and off individual error mitigation and suppression methods, including dynamical decoupling, gate and measurement twirling, measurement error mitigation, PEC, and ZNE. See [Error mitigation and suppression techniques](error-mitigation-and-suppression-techniques) for an explanation of each.

<Admonition type="note" title="Notes">
- Not all options are available for both primitives.   See the [available options table](runtime-options-overview#options-table) for the list of available options.
- Not all methods work together on all types of circuits.  See the [feature compatibility table](runtime-options-overview#options-compatibility-table) for details.
</Admonition>

<Tabs>
  <TabItem value="Estimator" label="Estimator">
    ```python
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

estimator = Estimator(backend)
options = estimator.options
# Turn on gate twirling.
options.twirling.enable_gates = True
# Turn on measurement error mitigation.
options.resilience.measure_mitigation = True

print(f">>> gate twirling is turned on: {estimator.options.twirling.enable_gates}")
print(f">>> measurement error mitigation is turned on: {estimator.options.resilience.measure_mitigation}")
```
  </TabItem>

  <TabItem value="Sampler" label="Sampler">
```python
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Estimator and Sampler have different options
sampler = Sampler(backend)
options = sampler.options

options.dynamical_decoupling.enable = True
# Turn on gate twirling. Requires qiskit_ibm_runtime 0.23.0 or later.
options.twirling.enable_gates = True

print(f">>> dynamical decoupling is turned on: {sampler.options.dynamical_decoupling.enable}")
print(f">>> gate twirling is turned on: {sampler.options.twirling.enable_gates}")

```
  </TabItem>
</Tabs>

## Turn off all error mitigation

For instructions to turn off all error mitigation, see the [Turn off all error suppression and mitigation](specify-runtime-options#no-error-mitigation) section.

## Next steps

<Admonition type="tip" title="Recommendations">
  - Walk through an example that uses error mitigation in the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
  - Learn more about [error mitigation and error suppression techniques.](error-mitigation-and-suppression-techniques)
  - [Configure error suppression.](configure-error-suppression)
  - Explore other [options.](runtime-options-overview)
  - Decide what [execution mode](execution-modes) to run your job in.
</Admonition>